In [7]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1, model="gpt-4-0125-preview")

# 디폴트 메모리키는 history 이기 때문에 지우고 다 그거로 대체해도된다.
memory = ConversationSummaryBufferMemory(llm=llm, max_token=80, memory_key="chat_history",
                                         return_messages=True)

# 얼마나 많은 메세지가 왔는 지 모르지만 MessagesPlaceholder(varible_name="chat_history"),
# 를 통해 중간을 대체한다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 과거 대화내용들을 기억해서 그 기억을 바탕으로 인간에게 대답을 해 주는 ai 야. 그 이전 대화내용들은 내가 프롬프트로 미리 넣어줄거야."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")

])

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]
    

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

# 이런 방식이 좀 복잡하지만, 모든 부분을 통제할 수 있어서 더 좋은 것 같다. - 노마드코더
def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question
        }
    )
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("내 이름은 조현호야.")

content='안녕하세요, 조현호님! 오늘 어떤 도움을 드릴까요?'


In [5]:
chain.invoke({
    "question": "내 이름은 뭘까?"
})

{'question': '내 이름은 뭘까?'}


AIMessage(content='이전 대화에서 당신의 이름에 대한 정보를 제공하지 않았기 때문에, 당신의 이름이 무엇인지 알 수 없습니다. 당신의 이름을 알려주시겠어요?')